In [3]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
vocabulary_size = 39261062

In [5]:
import os
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer

class BPE_token(object):
    def __init__(self):
        self.tokenizer = Tokenizer(BPE())
        self.tokenizer.normalizer = Sequence([
            NFC()
        ])
        self.tokenizer.pre_tokenizer = ByteLevel()
        self.tokenizer.decoder = ByteLevelDecoder()

    def bpe_train(self, paths):
        trainer = BpeTrainer(vocab_size=vocabulary_size, show_progress=True, inital_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>"
        ])
        self.tokenizer.train(paths, trainer)

    def save_tokenizer(self, location, prefix=None):
        if not os.path.exists(location):
            os.makedirs(location)
        self.tokenizer.model.save(location, prefix)

In [10]:
paths = ['sw-train.txt']

In [11]:
paths

['sw-train.txt']

In [12]:
tokenizer = BPE_token()
tokenizer.bpe_train(paths)
tokenizer.save_tokenizer('.')

In [13]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained('.')
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id,
  n_embd=64,
  n_layer=4,
  n_head=4
)
# creating the model
model = TFGPT2LMHeadModel(config)
model(model.dummy_inputs)
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 1865600   
 r)                                                              
                                                                 
Total params: 1,865,600
Trainable params: 1,865,600
Non-trainable params: 0
_________________________________________________________________


In [15]:
single_string = ''
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:
    x = f.read()
  single_string += x + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)

In [16]:
string_tokenized[:10]

[85, 467, 3638, 429, 64, 3771, 10, 64, 697, 742]

In [17]:
examples = []
block_size = 100
BATCH_SIZE = 128
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [18]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [ ]:
num_epoch = 10
history = model.fit(dataset, epochs=num_epoch) 

Epoch 1/10
464/464 [==============================] - 85s 184ms/step - loss: 6.8673 - accuracy: 0.1067
Epoch 2/10
464/464 [==============================] - 86s 185ms/step - loss: 6.8398 - accuracy: 0.1078
Epoch 3/10
464/464 [==============================] - 85s 183ms/step - loss: 6.8138 - accuracy: 0.1089
Epoch 4/10
464/464 [==============================] - 86s 185ms/step - loss: 6.7893 - accuracy: 0.1100
Epoch 5/10
464/464 [==============================] - 86s 185ms/step - loss: 6.7664 - accuracy: 0.1110


In [ ]:
text = "An bhfuil"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 50,
  num_beams = 5,
  temperature = 0.4,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)
print(tokenizer.decode(beam_output[0]))

In [ ]:
from transformers import WEIGHTS_NAME, CONFIG_NAME
model_to_save = model.module if hasattr(model, 'module') else model
output_model_file = os.path.join('.', WEIGHTS_NAME)
output_config_file = os.path.join('.', CONFIG_NAME)
# save model and model configs
model.save_pretrained('.')
model_to_save.config.to_json_file(output_config_file)
# save tokenizer
tokenizer.save_pretrained('.')

In [ ]:
# can then avoid retraining by loading these files when they exist:
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
model = TFGPT2LMHeadModel.from_pretrained(output_dir)